# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Описание данных
   
Для анализа представлен файл data.csv, содержащий следующую информацию:
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита 

## Шаг 1. Откройте файл с данными и изучите общую информацию

Открываем файл '/datasets/data.csv' и сохраняем его в переменной data. На экран выводим первые десять строк таблицы, а так же оцениваем объем данных:

In [1]:
import pandas as pd

data = pd.read_csv('/datasets/data.csv')
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод** Файл содержит 12 столбцов и 21525 сторк. 

В 2 столбцах указаны дробные числа, в 5 - целые, строки - в 5.


Для ответа на поставленные вопросы нас интересуют следующие графы:
- children — для проверки зависимости между наличием детей и возвратом кредита в срок;
- family_status — для проверки зависимости между семейным положением и возвратом кредита в срок;
- family_status_id — для проверки зависимости между семейным положением и возвратом кредита в срок;
- total_income — для проверки зависимости между уровнем дохода и возвратом кредита в срок;
- purpose — для проверки как разные цели кредита влияют на его возврат в срок.
- Графа debt содержит сведения о наличии задолженности по возврату кредитов.

Далее нам необходимо установить причины и обработать пропуски в графах 'days_employed'(общий трудовой стаж) и 'total_income'(ежемесячный доход), а также выявить и устранить дубликаты данных во всех рассматриваемых графах.

## Шаг 2. Предобработка данных

### Обработка пропусков

Предоложим, что пропуски в графах 'days_employed'(общий трудовой стаж) и 'total_income'(ежемесячный доход) в количестве 2174 строк относятся к клиентам, которые никогда не работали. Для проверки гипотезы посчитаем количество уникальных наименований в колонке "income_type"

In [3]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Не похоже, что в таблице есть 2174 никогда не работавших людей: в даных только 1 студент и 1 человек в декрете, безработных тоже всего 2. Гипотеза не подтвердилась, следовательно пропуски обусловлены иными причинами. Поскольку пропущенные данные носят количественный характер, заполним их характерными значениями с помощью метода median(). Данный метод лучше подходит для расчета стреднего дохода.

При этом, будем использовать группировку уровня дохода в зависимости от типа занятости (income type)/ 

Для начала проверим, действительно ли зависит средний доход от типа занятости.

In [4]:
#days_avg = data['days_employed'].mean()
#print('Заменяем пропуски в графе days_employed на значение:', days_avg)
#income_avg = data['total_income'].median()
#print('Заменяем пропуски в графе total_income на значение:', income_avg)
#data['days_employed'] = data['days_employed'].fillna(days_avg)
#data['total_income'] = data['total_income'].fillna(income_avg)

data.groupby('income_type')['total_income'].median()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Гипотеза подтвердилась, доход больше всего завист от графы с типом занятости. Заменим пропуски в зависимости от типа занятости с помощью функции и метода apply()

Пропуски в графе 'days_employed'(общий трудовой стаж) заполнять не будем, поскольку, как мы указали выше, данные этой графы для анализа нам не понадобятся. Удалим этот столбец

In [5]:
data.pop('days_employed')

data_grouped = data.groupby('income_type')['total_income'].median()
 
def func(row):
    if pd.isna(row['total_income']):
        return data_grouped.loc[row['income_type']]
    return row['total_income']
 
data['total_income'] = data.apply(func, axis=1)

Проверяем результат

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(5), object(5)
memory usage: 1.8+ MB


**Вывод** В графе 'total_income'(ежемесячный доход) пропуски заменены на характерные для данных столбцов значения 

### Замена типа данных

Графа total_income содержит вещественный формат данных. Для дальнейшего удобства переведем эти данные в целочисленный формат

In [7]:
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(6), object(5)
memory usage: 1.8+ MB


**Вывод** Для того, чтобы перевести строковые значения в числовой тип float64, используется метод Pandas —  to_numeric(). Нам же необходимо перевести вещественный тип в целочисленный, для этого используем метод astype()

### Обработка дубликатов

Считаем количество уникальных наименований в колонках 'education', 'family_status', 'gender', 'income_type', 'purpose' в таблице 'data' и выводим все уникальные наименования на экран с помощью метода .unique() и подсчета элементов в списке len()

In [8]:
unique_education = data['education'].unique()
print('Уникальных education:', len(unique_education), unique_education) 

unique_family_status = data['family_status'].unique()
print('Уникальных family_status:', len(unique_family_status), unique_family_status) 

unique_gender = data['gender'].unique()
print('Уникальных gender:', len(unique_gender), unique_gender) 

unique_income_type = data['income_type'].unique()
print('Уникальных income_type:', len(unique_income_type), unique_income_type) 

unique_purpose = data['purpose'].unique()
print('Уникальных purpose:', len(unique_purpose), unique_purpose)

Уникальных education: 15 ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
Уникальных family_status: 5 ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
Уникальных gender: 3 ['F' 'M' 'XNA']
Уникальных income_type: 8 ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
Уникальных purpose: 38 ['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой 

В графах education, family_status имеются дубликаты данных, вызванные несоблюдением регистра при их вводе. Применим метод str.lower() для устранения дубликатов

In [9]:
data['education'] = data['education'].str.lower()
print('Уникальных education:', len(unique_education), unique_education) 

data['family_status'] = data['family_status'].str.lower()
print('Уникальных family_status:', len(unique_family_status), unique_family_status) 

Уникальных education: 15 ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
Уникальных family_status: 5 ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']


Также необходимо устранить выявленные недочеты в графе 'gender', в которой кроме мужскго и женского пола указано XNA. Проанализируем количество таких данных с помощью метода value_counts():

In [10]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Учитывая, что такой случай всего один, он вызван, вероятнее всего, некорректным заполнением данных, поэтому присвоим этому значению с помощью replace() параметр F, поскольку женщин - заемщиков в рассматриваемой таблице в два раза больше чем мужчин.

In [11]:
data['gender'] = data['gender'].replace('XNA', 'F')
print('Уникальных gender:', len(unique_gender), unique_gender) 

Уникальных gender: 3 ['F' 'M' 'XNA']


Найдем дубликаты строк:

In [12]:
data.duplicated().sum()

71

Удалим дубликаты сток методом drop_duplicates()

In [13]:
data = data.drop_duplicates().reset_index(drop = True)
data.duplicated().sum()

0

Проверим уникальные значения в графе children

In [14]:
data['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Очевидно, что при вводе данных были допущены опечатки: 20 детей быть не может, поскольку даже 5 детей встречается всего в 5 случаях. Поэтому эти данные переименуем в 2. -1 ребенок так же опечатка, скорее сего связанная с неверным заполнением графы, переименуем ее на 1. 

In [15]:
#data['children'] = data['children'].replace(20, 2)
#data['children'] = data['children'].replace(-1, 1)
data.loc[data['children'] == 20, 'children'] = 2
data.loc[data['children'] == -1, 'children'] = 1

data['children'].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

**Вывод** Удалены дубликаты строк, дубликаты в графах "образование" и "количество детей"

### Лемматизация

В ходе анализа данных выявлены дубликаты данных в графе purpose, устранить которые можно только с помощью категоризации. . Для этого используем лемматизацию с помощью библиотеки PyMystem. Ранее созданный список unique_purpose преобразуем в строку методом join(), поскольку PyMystem работает только с данными в виде строки

In [16]:
from pymystem3 import Mystem
m = Mystem()

lem_purpose = ' '.join(unique_purpose)

lemmas = m.lemmatize(lem_purpose)

from collections import Counter
print(Counter(lemmas)) 

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


**Вывод** Основные цели получения кредита - недвижимость, автомобиль, образование, свадьба

### Категоризация данных

В результате леммитизации мы установили 4 основные цели получения кредита. Применим эти категории к графе purpose с помощью функции и метода apply, создав графу purpose_id. Проверим полученный результат с помощью value_counts()

In [17]:
def purpose_id(purpose):
    if 'недвиж' in purpose:
        return 'недвижимость'
    if 'жил' in purpose:
        return 'недвижимость'
    if 'авто' in purpose:
        return 'автомобиль'
    if 'образ' in purpose:
        return 'образование'
    if 'свадьб' in purpose:
        return 'свадьба'
    
data['purpose_id'] = data['purpose'].apply(purpose_id)
    
data['purpose_id'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_id, dtype: int64

Чтобы ответить на вопрос о влиянии уровня дохода на возврат кредита, создадим категории для графы total_income. Для этого сначала узнаем максимальное и минимальное значение данной графы

In [18]:
data['total_income'].max()

2265604

In [19]:
data['total_income'].min()

20667

Учитывая, что ранее мы вычислили медианный средний доход как 145018 создадим три категории дохода заемщиков: небольшой - 0 (до 100000), средний - 1 (от 100000 до 200000), высокий - 2 (более 200000). 

Поскольку чаще всего разделяют людей по доходам на три категории - люди с невысокими доходами, средний клас и люди с высокими доходами, опираясь на полученные нами данные - минимальный доход - 20677, средний (медианный) - 145018 и самый высокий - 2265604, мы также будем использовать эти три категории в исследовании. Границы категорий в 100000 и 200000 возьмем условно, отталкиваясь от медианного среднего дохода в 145018

In [20]:
def total_income_id(total_income):
    if total_income < 100000:
        return 0
    if 200000 > total_income >= 100000:
        return 1
    if total_income >= 200000:
        return 2
    
data['total_income_id'] = data['total_income'].apply(total_income_id)
    
data['total_income_id'].value_counts()

1    11924
2     5067
0     4463
Name: total_income_id, dtype: int64

In [21]:
#Для ответа на вопрос о влиянии количества детей на возврат кредита создадим 2 категории для графы children - заемщики с детьми (1) и без детей (0)

#def children_id(children):
#    if children == 0:
#        return 0
#    else:
#        return 1
#    
#data['children_id'] = data['children'].apply(children_id)
#    
#data['children_id'].value_counts()

**Вывод** Создали категории для граф [purpose] и [total_income] для расчета коэффициентов по заданным вопросам

Одним из заданий к проекту является вопрос "перечислите, какие «словари» вы выделили для этого набора данных, и объясните, почему".

В представленной таблице представлены два параметра - к столбцам [education] - education_id и к [family_status] - family_status_id.

Данные параметры удобно использовать как ключи в словарях, в которых ключ — это название столбца, к которому нужно применить функции, а значением выступает список с названиями функций.

Возьмем ключи  education_id и family_status_id и применинм функции sum и median к значениям из total_income

In [22]:
data.groupby('education_id').agg({'total_income': ['count', 'median']})

#data.pivot_table(index=['education', 'education_id'], columns='children', values='total_income', aggfunc='mean').fillna(0)

total_income          
                    count    median
education_id                       
0                    5250  171485.0
1                   15172  141695.0
2                     744  155759.0
3                     282  118941.5
4                       6  157259.5

In [23]:
data.groupby('family_status_id').agg({'total_income': ['count', 'median']})

#data.pivot_table(index=['family_status', 'family_status_id'], columns='gender', values='total_income', aggfunc='count').fillna(0)

total_income        
                        count  median
family_status_id                     
0                       12339  142969
1                        4151  142594
2                         959  127310
3                        1195  143450
4                        2810  142594

Как видим, если образование заметно влияет на медианный доход, то семейный статус на него практически не оказывает влияния

Словари также создают для оптимизации многократно повторяющихся данных:

In [24]:
family_dict = data[['family_status', 'family_status_id']]

family_dict = family_dict.drop_duplicates().reset_index(drop=True)

display(family_dict.sort_values('family_status_id'))

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


In [25]:
education_dict = data[['education', 'education_id']]

education_dict = education_dict.drop_duplicates().reset_index(drop=True)

display(education_dict.sort_values('education_id'))

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Графа debt отвечает на впорос - имел ли клиент задолженность по возврату кредитов: 0 - нет (False), 1 - да (True). Определим коэффициент зависимости между наличием детей и возвратом кредита в срок как отношение количества невозвратов кредита к общему количеству кредитов для каждой из категории заемщиков. Построим сводную таблицу с помощью метода pivot_table. Значение values возьмем из любой из неиспользуемых в функции ячеек для подстчета количества значений. Чем выше получаемый коэффициент, тем больше невозвратов кредита в срок.

In [26]:
data_pivot = data.pivot_table(index=['children'], columns='debt', values='total_income', aggfunc='count')
data_pivot['ratio'] = data_pivot[1] / (data_pivot[0] + data_pivot[1])

data_pivot = data_pivot.fillna(0)
display(data_pivot.sort_values(by = 'ratio', ascending = False))

debt,0,1,ratio
children,,,
4,37.0,4.0,0.097561
2,1926.0,202.0,0.094925
1,4410.0,445.0,0.091658
3,303.0,27.0,0.081818
0,13028.0,1063.0,0.075438
5,9.0,0.0,0.000000


**Вывод** Семьи с детьми чаще нарушают сроки возврата кредита. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Аналогичным образом рассчитаем коэффициент зависимости между семейным положением и возвратом кредита в срок

In [27]:
data_pivot = data.pivot_table(index=['family_status'], columns='debt', values='total_income', aggfunc='count')
data_pivot['ratio'] = data_pivot[1] / (data_pivot[0] + data_pivot[1])

display(data_pivot.sort_values(by = 'ratio', ascending = False))

debt,0,1,ratio
family_status,,,
не женат / не замужем,2536,274,0.097509
гражданский брак,3763,388,0.093471
женат / замужем,11408,931,0.075452
в разводе,1110,85,0.071130
вдовец / вдова,896,63,0.065693


**Вывод** Люди, которые не имеют или не имели оформленных брачных отношений, чаще нарушают сроки возварта кредита

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Рассчитаем кофэффициент зависимости между уровнем дохода и возвратом кредита в срок. Ранее мы расчитали уровни дохода как небольшой - 0 (до 100000), средний - 1 (от 100000 до 200000), высокий - 2 (более 200000):

In [28]:
data_pivot = data.pivot_table(index=['total_income_id'], columns='debt', values='total_income', aggfunc='count')
data_pivot['ratio'] = data_pivot[1] / (data_pivot[0] + data_pivot[1])

display(data_pivot.sort_values(by = 'ratio', ascending = False))

debt,0,1,ratio
total_income_id,,,
1,10895,1029,0.086297
0,4109,354,0.079319
2,4709,358,0.070653


**Вывод** Люди со средним доходом менее дисциплинированы в вопросах возврата кредита в срок

- Как разные цели кредита влияют на его возврат в срок?

Рассчитаем кофэффициент зависимости целей кредита на его возврат в срок:

In [29]:
data_pivot = data.pivot_table(index=['purpose_id'], columns='debt', values='total_income', aggfunc='count')
data_pivot['ratio'] = data_pivot[1] / (data_pivot[0] + data_pivot[1])

display(data_pivot.sort_values(by = 'ratio', ascending = False))

debt,0,1,ratio
purpose_id,,,
автомобиль,3903,403,0.093590
образование,3643,370,0.092200
свадьба,2138,186,0.080034
недвижимость,10029,782,0.072334


**Вывод** Клиенты, берущие кредит на образование и автомобиль чаще нарушают сроки погашений по кредитам, чем те, кто взял кредит на недвижимость и свадьбу.

## Шаг 4. Общий вывод

Семьи с детьми чаще нарушают сроки возврата кредита, чем те, кто не имеет детей.

Люди, которые не имеют или не имели оформленных брачных отношений, чаще нарушают сроки возварта кредита. Люди, официально состоящие в брачных отношениях, вдовцы и разведенные существенно боле дисциплинированы в этом вопросе.

Люди со средним доходом менее дисциплинированы в вопросах возврата кредита в срок. Меньше всего нарушают сроки погашения кредита люди с уровнем дохода более 200000

Клиенты, берущие кредит на образование и автомобиль чаще нарушают сроки погашений по кредитам, чем те, кто взял кредит на  свадьбу. Наименьшее количество нарушений сроков по кредиту допускают заемщики, взявшие кредит на жилье и недвижимость